# A implementation of Sequence-Based Behavior Group Clustering Algorithm


In [21]:
import pickle
import os

In [22]:
% run Alignment_Fast3.ipynb
% run StructMatchGap3.ipynb
% run StageMatrix.ipynb
% run Motif.ipynb
% run OutputStage.ipynb
% run CommonMotifAnalysis_Tmp.ipynb

# Doing global alignment and Calculate common motif.
# will return a common motif dict
def do_globalAlignment(rep1, rep2):
    # Aligment
    align_dict = dict()
    BASE = "rep1"
    align_dict['rep1'] = pairwise_NW( rep1, rep1, 2, -1, -3, 1)[2]
    align_dict['rep2'] = pairwise_NW( rep1, rep2, 2, -1, -3, 1)[2]
    
    # get 'Match Matrix' and 'Gap List'
    matchMatrix, gapSeqList = structMatchGap(align_dict, BASE)
    stageMatrixResult = stageMatrix(matchMatrix, gapSeqList)
    Motif_Obj = Motif(stageMatrixResult, BASE)
    outputStage = OutputStage(stageMatrixResult, None, BASE, Motif_Obj)
    
    executionTrace_dict = {"rep1":rep1, "rep2":rep2}
    
    commonMotif = CommonMotif(stageMatrixResult, Motif_Obj, executionTrace_dict, outputStage)
    
    # comMotifdict= {'s<stage>_<motif>': [CMS], oriIdxRange1, oriIdxRange2},
    comMotif_dict = commonMotif.getComMotifDict()  
    return comMotif_dict

In [23]:
def removeDuplicateAPI(featureTrace): # remove duplicate api if continuously occur
    result = []
    lastAPI = ""
    for api in featureTrace:
        if lastAPI != api: # find new api
            result.append(api)
            lastAPI = api
    return result

def removeUnwantedAPI(featureTrace): # remove unwanted api
    result = []
    unwanted_api = {'CloseHandle', 'OpenThread', 'RegOpenKey', 'RegCloseKey'}
    frequently_used_lib = {'imm32', 'lpk', 'gdi32', 'kernel32', 'ntdll', 'user32', 'comctl32', 'advapi64'}

    for api in featureTrace:
        API = api.split('#')[0]
        
        if API == "LoadLibrary": # api is LoadLibrary
            libName = api.split("@")[2]
            if libName not in frequently_used_lib: # found new library, add it into lib_set and result_Hooklog
                result.append(api)
                frequently_used_lib.update(libName)
                
        elif API not in unwanted_api: # api not unwanted
            result.append(api)
            
    return result

In [24]:
% run FeatureTrace.ipynb
#******************** the output toMergeCandidate_Dict have to change to set

# initialize all traces as "to merge candidates clusters"
def initialCandidateDict(data_directory):
    
#     toMergeCandidate_List = [] #list()
    toMergeCandidate_Dict = dict()
    
    # get feature hooklogs
    FeatTrace = FeatureTrace
    traceName_list = [f for f in os.listdir(data_directory) if f.endswith('.trace.hooklog') ] # py36_leoqaz12
    ft_count = 0
    for traceName in traceName_list:
        featureTrace = FeatTrace(data_directory + traceName).getTrace_noContainTS()
#         featureTrace = [line.rstrip('\n') for line in open(data_directory + traceName)] # use txt as featureTrace directly
        featureTrace = removeDuplicateAPI(featureTrace)    
        featureTrace = removeUnwantedAPI(featureTrace)
        clusterName = "G"+str(ft_count)
        # R = tuple( clusterName, list(  tuple(featureTrace, fTStartIdx, fTEndIdx) ) ), the representative of cluster.
        R = (clusterName, [(featureTrace, 0, len(featureTrace)-1)] )
        clusterMembers = set()
        traceName = shortenHooklogName(traceName)
        clusterMembers.add(traceName)
        
        toMergeCandidate_Dict[ft_count] = (R, clusterMembers)
        
        ft_count+=1
        
#     print("-- Finish Initializing --")
    return toMergeCandidate_Dict
#     return toMergeCandidateSet

In [25]:
# shorten Name to first 6 charactors
def shortenHooklogName(traceName):
    hashValue = traceName[0:6]
    pid = traceName.split("_")[1].split(".")[0]
    return hashValue+"_"+pid

In [26]:
# input: two R
# output: new RepresentativeR of inputs;
def get_Representative(Ri, Rj):
    rep1 = [] #list()
    rep2 = [] #list()

#     print(Ri[0], Rj[0])
    for i in range(len(Ri[1])): # get length of R's common motif seqs  (p.s. Ri[0] is clusterName)
        rep1 += Ri[1][i][0]
    for i in range(len(Rj[1])):
        rep2 += Rj[1][i][0]
    
    repNew = [] #list() 
    
    if(rep1 and rep2):
        comMotif_dict = do_globalAlignment(rep1, rep2) # do Alignment
        newStartIdx = 0

        for m in sorted(comMotif_dict.keys(), key = lambda x : int(x.split('_')[0][1:])): # sorted by stages
            cmsList = comMotif_dict[m]
            newEndIdx = newStartIdx + len(cmsList[0]) - 1
            repNew.append((cmsList[0], newStartIdx, newEndIdx, cmsList[1], cmsList[2]))
                      # [CMS, newCMSStartIdx, newCMSEndIdx, oriIdxRange1, oriIdxRange2]
            newStartIdx = newEndIdx + 1

    return repNew

In [27]:
# return a dictionary that contains the initializing informations
#
# initialDict = {clusterName : (originalName, initialLength)}

def getInitialDict(toMergeCandidateDict):
    initialDict = dict()
    for key, value in toMergeCandidateDict.items():
        clusterName = value[0][0]
        initialLen = value[0][1][0][2] + 1
        originalName = value[1].pop()
        initialDict[clusterName] = (originalName, initialLen)
        value[1].add(originalName)
    return initialDict

In [28]:
# return a dict that contains only original name
# nameDict = {clusterName: original name}

def getInitialNameDict(initialDict):
    nameDict = dict()
    for key, value in initialDict.items():
        name = value[0]
        nameDict[key] = name
    return nameDict

In [29]:
import functools

# compute score of Rnew
# the score calculate method is the length ratio of new to origin one

# Ri is a tuple like ('G0', [[['A#A', 'C#C'], 0, 1, (0, 1), (1, 2)]])
def compute_Score(Ri, Rj, Rnew):
    if(Rnew[1]):
        L_Ri = functools.reduce(lambda x,y:x+y, [(i[2]-i[1]+1) for i in Ri[1]])
        L_Rj = functools.reduce(lambda x,y:x+y, [(j[2]-j[1]+1) for j in Rj[1]])
    
        Lorg = max(L_Ri, L_Rj)
        Lnew = functools.reduce(lambda x,y:x+y, [(n[2]-n[1]+1) for n in Rnew[1]]) 
        return float(Lnew)/Lorg
    else:
        return 0

In [30]:
# def snippetPacketChunking(toMergeCandidateDict, outputPath, roundCounter):
#     if not os.path.isdir(outputPath): os.makedirs(outputPath)
#     dictKeys = list(toMergeCandidateDict.keys())
    
#     snippetPacketDict = dict()
#     snippetPacket = []

#     print('Total Grams:',len(dictKeys))
#     packetCount = 0
#     pairCount = 0
#     duplicateFlag = False
#     for i in range(len(dictKeys)):
#         for j in range(i+1, len(dictKeys)):
#             pair = PairwisePair(toMergeCandidateDict[dictKeys[i]], toMergeCandidateDict[dictKeys[j]])
#             snippetPacket.append(pair)
#             pairCount += 1
#             duplicateFlag = False
#             if(len(snippetPacket) >= 10000):
#                 snippetPacketDict[packetCount] = snippetPacket
#                 snippetPacket = []
#                 packetCount += 1
#                 duplicateFlag = True # avoid loop end after leaving this if.

#     if(not duplicateFlag):
#         snippetPacketDict[packetCount] = snippetPacket

#     print("Total Pairs:", pairCount)
#     print("Divided into ", len(snippetPacketDict.keys()), "chunks")
    
#     with open(outputPath + 'SnippetPacket_round'+str(roundCounter)+'.pickle', 'wb') as handle:
#         pickle.dump(snippetPacketDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
# def findMergeCandidateScoreListOfSnippet(snippetPacket, generatedSeqNum, pickleDir, snippetIndex):
#     if not os.path.isdir(pickleDir): os.makedirs(pickleDir)
#     scoreList = [] #list()
    
#     for pair in snippetPacket:
#         Ri = pair.getROfClusterI()
#         Rj = pair.getROfClusterJ()
#         repNew = get_Representative(Ri, Rj)
#         clusterTempName = "G" + str(generatedSeqNum)
#         Rnew = (clusterTempName , repNew)
            
#         # compute merge score of Rnew
#         score = compute_Score(Ri, Rj, Rnew)
#         Ri_name = Ri[0]
#         Rj_name = Rj[0]
#         scoreList.append((score, Rnew, Ri_name, Rj_name))
#         Ri = None
#         Rj = None
#         repNew = None
#         Rnew = None
    
#     print("ScoreList Len : ", len(scoreList))

#     with open(pickleDir + str(snippetIndex) +"tmp.pickle", 'wb') as handle:
#         pickle.dump(scoreList, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     scoreList = None
        
# def concateSnippetScoreList(tmpPickleDir):
#     scoreList = []
#     pList = os.listdir(tmpPickleDir)
#     print("tmp Pickle numbers = " , len(pList))
#     for file in pList:
#         with open(tmpPickleDir + file, 'rb') as inputFile:
#             tmpList= pickle.load(inputFile)
#             scoreList.extend(tmpList)
#             print("TempList Len : ",len(tmpList))

#     return sorted(scoreList, key=lambda tup:tup[0], reverse=True) # sorting by score (from biggest to smallest) 

In [31]:
# get score list of toMergeCandidateDict(single iteration) from highest to lowest

def findMergeCandidateScoreList(toMergeCandidateDict, generatedSeqNum):
    scoreList = [] #list()
    dictKeys=[]
#     del(list)
    for k in toMergeCandidateDict:
        dictKeys.append(k) #leoqaz12
#     try:
#         dictKeys = list(toMergeCandidateDict.keys()) #original
#     except:
#         del(list)
#         dictKeys = list(toMergeCandidateDict.keys())
    
    sensitiveAPIs = {"CreateProcessInternal", "OpenProcess", "WinExec", "CreateThread", "OpenThread", "CreateRemoteThread",
                     "CopyFile", "CreateFile", "WriteFile", "ReadFile", "DeleteFile", "RegCreateKey", "RegSetValue",
                     "InternetOpen", "InternetConnect", "HttpSendRequest", "WinHttpOpen", "WinHttpSendRequest", "WinHttpWriteData", "WinHttpCreateUrl"}
    
    for i in range(len(dictKeys)):
        for j in range(i+1, len(dictKeys)):
            
            # toMergeCandidateDict[i][1] is memberSet
            Ri = toMergeCandidateDict[ dictKeys[i] ][0] # Ri is a tuple like ('G0', [[['A#A', 'C#C'], 0, 1, (0, 1), (1, 2)]])
            Rj = toMergeCandidateDict[ dictKeys[j] ][0]
#             print(toMergeCandidateDict[ dictKeys[i] ][1], toMergeCandidateDict[ dictKeys[j] ][1])
            repNew = get_Representative(Ri, Rj)
            clusterTempName = "G" + str(generatedSeqNum)
            Rnew = (clusterTempName , repNew)

            score = compute_Score(Ri, Rj, Rnew)
            Ri_name = Ri[0]
            Rj_name = Rj[0]
            scoreList.append((score, Rnew, Ri_name, Rj_name))
            Ri=None
            Rj=None
            Rnew=None
            repNew=None
#             else:
#                 print("Rep Sequence Length smaller than 26! Length: ", RnewSequenceLen)

    if(len(scoreList) > 0):
        scoreList.sort(key=lambda tup:tup[0], reverse=True) # sorting by score (from biggest to smallest) 
#         print("ScoreList Length in method : ", len(scoreList))
    else:
        print("No common motif")
    
    return scoreList # list = [(score, Rnew, Ri_name, Rj_name), (score, Rnew, Ri_name, Rj_name), ...]

In [32]:
def checkExactlySameCandidates(scoreList):
    globalPoolDict = dict() # a dict contains many sets.  dict = {index0: memberSet, 1: memberSet, 2:...}
    newScoreList = [] #list() # list = [(score, R, memberSet), (score, R, memberSet), ...]
    scoreListIdx = 0
    for rank in scoreList:
        score = rank[0]
       
        if(score == 1.0):
            
            Ri_name = rank[2]
            Rj_name = rank[3]
            
            duplicate = False
            for key, memberSet in globalPoolDict.items():
                if(Ri_name in memberSet) or (Rj_name in memberSet):
                    memberSet.add(Ri_name)
                    memberSet.add(Rj_name)
                    
                    # update newScoreList 'memberSet' element
                    newScoreList[key] = (newScoreList[key][0], newScoreList[key][1], memberSet)
                    duplicate = True
                    
            # Find new independent pair, add into newScoreList and create new dict key
            if(duplicate is False):
                memberSet = set()
                memberSet.add(Ri_name)
                memberSet.add(Rj_name)
                globalPoolDict[scoreListIdx] = memberSet
                
                Rnew = rank[1]
                newScoreList.append((score, Rnew, memberSet))
                scoreListIdx += 1
        else:
            Rnew = rank[1]
            Ri_name = rank[2]
            Rj_name = rank[3]
            memberSet = set()
            memberSet.add(Ri_name)
            memberSet.add(Rj_name)
            newScoreList.append((score, Rnew, memberSet))
            scoreListIdx += 1
    globalPoolDict = None
    return newScoreList # list = [(score, R, memberSet), (score, R, memberSet), ...]
        

In [33]:
# # unit test
# item1 = (1.0, ("G0", "[['A#A', 'B#B','B#B', 'C#C','D#D'], 0, 2]"), "a.txt", "b.txt")
# item2 = (1.0, ("G1", "[['A#A', 'B#B','B#B', 'C#C','D#D'], 0, 2]"), "a.txt", "c.txt")
# item3 = (1.0, ("G2", "[['A#A', 'B#B','B#B', 'C#C','D#D'], 0, 2]"), "b.txt", "c.txt")
# item4 = (1.0, ("G3", "[['A#A', 'B#B','B#B', 'C#C','D#D'], 0, 2]"), "c.txt", "d.txt")
# item5 = (1.0, ("G4", "[['E#A', 'F#B'], 0, 2]"), "e.txt", "f.txt")
# item6 = (0.8, ("G5", "[['X#A', 'Y#B'], 0, 2]"), "x.txt", "y.txt")

# scoreList = [item1, item2, item3, item4, item5, item6]

# newScoreList = checkExactlySameCandidates(scoreList)
# print(newScoreList)

In [34]:
# add Rnew into toMergeCandidateDict and remove member of Rnew from candidates.

def mergeCandidateClusters_new(toMergeCandidateDict, intermediatePoolDict, scoreList, generatedSeqNum, initialDict, definedThreshold):
    initialNameDict = getInitialNameDict(initialDict) # get original name for reference in output.
    
    currentMergedSet = set()
    for rank in scoreList:
        score = rank[0]
        memberSet = rank[2] # memberSet of highest score

        # the minmum score this round is smaller than threshold
        if(score < definedThreshold):
            break
        
        exclusiveness = False
        
        # check exclusiveness
        for member in memberSet:
            if(member in currentMergedSet):
                exclusiveness = True
                break
                
        if(not exclusiveness):
            clusterMembers = set() # create cluster member set with original Name
            for member in memberSet:
                nameOfMember = int(member.split('G')[1])
                del toMergeCandidateDict[nameOfMember]
                
                if member in initialNameDict:
                    clusterMembers.add(initialNameDict[member])
                else:
                    clusterMembers.add(member)
                    
                # Mark elements are merged
                currentMergedSet.add(member) # update currentMergedSet
            
            Rnew = rank[1][1] # representative without old clusterName (i.e., rank[1] = (Name, Rep.))
            newName = "G" + str(generatedSeqNum)
            new_Cluster = (newName, Rnew)
            
            toMergeCandidateDict[generatedSeqNum] = (new_Cluster, clusterMembers)
            intermediatePoolDict[generatedSeqNum] = (score, new_Cluster, clusterMembers) # (score, newCluster, members)
            generatedSeqNum += 1
    currentMergedSet = None
    return toMergeCandidateDict, intermediatePoolDict, generatedSeqNum

In [35]:
# add Rnew into toMergeCandidateDict and remove member of Rnew from candidates.

def mergeCandidateClusters(toMergeCandidateDict, intermediatePoolDict, scoreList, generatedSeqNum, initialDict):
    currentMergedSet = set()
    
    initialNameDict = getInitialNameDict(initialDict)
    
    for rank in scoreList:
        Ri_name = rank[2] # member1 of highest score
        Rj_name = rank[3] # member2 of highest score
        
        # check exclusiveness that candidate have been merged in current scoreList.
        # if both two element haven't been processed then create new cluster.
        if((Ri_name not in currentMergedSet) and (Rj_name not in currentMergedSet)):
            # remove candidates in @toMergeCandidateDict
            keyOfRi = int(Ri_name.split('G')[1])
            keyOfRj = int(Rj_name.split('G')[1])
            del toMergeCandidateDict[keyOfRi], toMergeCandidateDict[keyOfRj]

            Rnew = rank[1] # get representative of highest score
            newName = "G" + str(generatedSeqNum) # update clusterName
        
            new_Cluster = (newName, Rnew[1])

            clusterMembers = set() # create cluster member set
            if Ri_name in initialNameDict:
                clusterMembers.add(initialNameDict[Ri_name])
            else:
                clusterMembers.add(Ri_name)
            
            
            if Rj_name in initialNameDict:
                clusterMembers.add(initialNameDict[Rj_name])
            else:
                clusterMembers.add(Rj_name)
            
            
            toMergeCandidateDict[generatedSeqNum] = (new_Cluster, clusterMembers)
            intermediatePoolDict[generatedSeqNum] = (rank[0], new_Cluster, clusterMembers) # (score, newCluster, members)

            generatedSeqNum += 1
        
        # Mark elements are merged
        currentMergedSet.add(Ri_name) # update currentMergedSet
        currentMergedSet.add(Rj_name)
        
    return toMergeCandidateDict, intermediatePoolDict, generatedSeqNum

In [36]:
def clusterInitializedReps_semi(initializedReps_dict, tag, outputPath, thresholdValue):
    intermediatePool = dict()
    roundInfos = dict()
    residual = None # used to save residual candidate when algorithm stop.

    toMergeCandidateDict = initializedReps_dict # using residualRepsDict as toMergeCandidateDict (skip initialization)

    # initialDict = {clusterName : (originalName, initialLength)}
    initialDict = getInitialDict(toMergeCandidateDict)
    
    
    roundProduct = [] #list()
    for key, value in initialDict.items():
        roundProduct.append(key)
    roundInfos[0] = roundProduct # record product in round 0 (i.e., initialization)
    
#     generatedSeqNum = len(toMergeCandidateDict) # counter after initialize. Used to naming clusters.
    generatedSeqNum = 818

#     print("-- Start Clustering --")
#     print("Threshold set =", thresholdValue)
    roundCounter = 1
    
    lastBreakPoint = 0
    
    if(lastBreakPoint == 0):
        with open(outputPath+'Pickle/' + 'tmpInit.pickle', 'wb') as f:
            pickle.dump(initialDict, f, protocol=pickle.HIGHEST_PROTOCOL)
        with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(1)+'.pickle', 'wb') as handle:
            pickle.dump(toMergeCandidateDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(outputPath+'Pickle/' + 'tmpInit.pickle', 'rb') as f:
            initialDict = pickle.load(f)
            
    
    while(1):
#         print("Current Round : Round ", roundCounter)
        if(roundCounter >= lastBreakPoint):

            with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(roundCounter)+'.pickle', 'rb') as mHandle:
                toMergeCandidateDict = pickle.load(mHandle)
            if(roundCounter != 1):
                with open(outputPath+ "pickle/" + 'roundInfos_round'+str(roundCounter-1)+'.pickle', 'rb') as rHandle:
                    roundInfos = pickle.load(rHandle)
                with open(outputPath+ "pickle/" + 'intermediate_round'+str(roundCounter-1)+'.pickle', 'rb') as iHandle:
                    intermediatePool = pickle.load(iHandle)


            if(len(toMergeCandidateDict) == 1):
                residual = toMergeCandidateDict # output residual candidates.
                break

            if(roundCounter != lastBreakPoint):
                snippetPacketChunking(toMergeCandidateDict, outputPath+'SnippetPickle/', roundCounter)

            with open(outputPath +'SnippetPickle/' + 'SnippetPacket_round'+str(roundCounter)+'.pickle', 'rb') as handle:
                spDict = pickle.load(handle)

            if(roundCounter == lastBreakPoint):
                # calculate scoreList in candidate clusters
                for snippetIndex in range(1, len(spDict.keys())):
                    print('scoring snippet :' , snippetIndex)
                    snippetPacket = spDict[snippetIndex]
                    findMergeCandidateScoreListOfSnippet(snippetPacket,
                                                         generatedSeqNum,
                                                         outputPath+str(roundCounter)+'TmpPickle/',
                                                         snippetIndex)
            else:
                for snippetIndex, snippetPacket in spDict.items():
                    # if snippetIndex==5:break
                    print('scoring snippet :' , snippetIndex)
                    findMergeCandidateScoreListOfSnippet(snippetPacket,
                                                         generatedSeqNum
                                                         , outputPath+str(roundCounter)+'TmpPickle/',
                                                         snippetIndex)
            # break        
            print("-- Finish scoring --")
            scoreList = concateSnippetScoreList(outputPath+str(roundCounter)+'TmpPickle/')
            print("-- Finish concatenating scoreList --")
#             print("total ScoreList Length : ", len(scoreList))

            # check and merge exactly the same candidates before merge clusters
            scoreList = checkExactlySameCandidates(scoreList)
            print("-- Finish checking 100% same candidates --")
            
            with open(outputPath+str(roundCounter)+'TmpPickle/' + 'scoreList.pickle', 'wb') as sListHandle:
                pickle.dump(scoreList, sListHandle, protocol=pickle.HIGHEST_PROTOCOL)
            
            # generated Clusters in This Round:
            nameIdxStart = generatedSeqNum
            
            toMergeCandidateDict, intermediatePool, generatedSeqNum = mergeCandidateClusters_new(
                toMergeCandidateDict, intermediatePool, scoreList, generatedSeqNum, initialDict, thresholdValue)
            print("-- Finish merging clusters --")

#             print("generatedSeqNum now: ", generatedSeqNum)

            # check if algorithm should stop when merge score under threshold
            # if a score smaller than threshold, then it will break out when merging.
            # Hense, if the 'generatedSeqNum' equals than 'nameIdxStart', means that no any new generated cluster.
            # (if occurr a new cluster, generatedSeqNum will add one.)
            if(generatedSeqNum == nameIdxStart):
                residual = toMergeCandidateDict # output residual candidates.
                break # end algorithm
            
            nameIdxEnd = generatedSeqNum
            
            # Record clusters generated in this round
            for idx in range(nameIdxStart, nameIdxEnd):
                if roundInfos.get(roundCounter) is None:
                    roundProduct = [] #list()
                    roundProduct.append(intermediatePool[idx][1][0])
                    roundInfos[roundCounter] = roundProduct
                else:
                    roundInfos[roundCounter].append(intermediatePool[idx][1][0])
                    
            roundCounter += 1

            with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(roundCounter)+'.pickle', 'wb') as mHandle:
                pickle.dump(toMergeCandidateDict, mHandle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(outputPath+ "pickle/" + 'intermediate_round'+str(roundCounter-1)+'.pickle', 'wb') as iHandle:
                pickle.dump(intermediatePool, iHandle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(outputPath+ "pickle/" + 'roundInfos_round'+str(roundCounter-1)+'.pickle', 'wb') as rHandle:
                pickle.dump(roundInfos, rHandle, protocol=pickle.HIGHEST_PROTOCOL)


        else:
            roundCounter += 1


    print("-- Finish Clustering --")

    return intermediatePool, initialDict, roundInfos, residual

In [37]:
### Main Function of SBBGCA ###

import pickle

def do_SBBGCA_clustering(data_directory, tag, outputPath, thresholdValue):
    testDict = {0: (('G0', [[['A#A', 'B#B','B#B', 'C#C','D#D'], 0, 2]]),{"a.trace.hooklog"}),
                1:(('G1', [[['A#A','B#B','C#C','D#D',"G#G"], 0, 2]]),{"b.trace.hooklog"}),
                   2:(('G2', [[["B#B",'F#F','C#C','D#D', 'G#G'], 0, 2]]),{"c.trace.hooklog"}),
                      3:(('G3', [[['Q#Q','C#C','D#D','G#G','M#M'], 0, 2]]),{"d.trace.hooklog"}),
                           4:(('G4', [[['A#A','Q#Q','C#C','G#G','M#M'], 0, 2]]),{"e.trace.hooklog"})}
    intermediatePool = dict()
    roundInfos = dict()
    residual = None # used to save residual candidate when algorithm stop.
#     toMergeCandidateDict = testDict
    toMergeCandidateDict = initialCandidateDict(data_directory) # initialize @toMergeCandidateDict

    # initialDict = {clusterName : (originalName, initialLength)}
    initialDict = getInitialDict(toMergeCandidateDict)
    
    roundProduct = [] #list()
    for key, value in initialDict.items():
        roundProduct.append(key)
    roundInfos[0] = roundProduct # record product in round 0 (i.e., initialization)
    
    generatedSeqNum = len(toMergeCandidateDict) # counter after initialize. Used to naming clusters.

#     print("-- Start Clustering --")
#     print("Threshold set =", thresholdValue)
    roundCounter = 1
#     generatedSeqNum = 224
#     roundCounter = 3
    try:
        if(roundCounter != 1):
            with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(roundCounter)+'.pickle', 'rb') as mHandle:
                toMergeCandidateDict = pickle.load(mHandle)
            with open(outputPath+ "pickle/" + 'roundInfos_round'+str(roundCounter-1)+'.pickle', 'rb') as rHandle:
                roundInfos = pickle.load(rHandle)
            with open(outputPath+ "pickle/" + 'intermediate_round'+str(roundCounter-1)+'.pickle', 'rb') as iHandle:
                intermediatePool = pickle.load(iHandle)

        while(1):
            print("Round: ", roundCounter)
            if(len(toMergeCandidateDict) == 1):
                residual = toMergeCandidateDict # output residual candidates.
                break

            # calculate scoreList in candidate clusters
            scoreList = findMergeCandidateScoreList(toMergeCandidateDict, generatedSeqNum)

            # check and merge exactly the same candidates before merge clusters
            scoreList = checkExactlySameCandidates(scoreList)

            import os
            if not os.path.isdir(outputPath+str(roundCounter)+'TmpPickle/'):
                os.makedirs(outputPath+str(roundCounter)+'TmpPickle/')

            with open(outputPath+str(roundCounter)+'TmpPickle/' + 'scoreList.pickle', 'wb') as sListHandle:
                    pickle.dump(scoreList, sListHandle, protocol=pickle.HIGHEST_PROTOCOL)

            # generated Clusters in This Round:
            nameIdxStart = generatedSeqNum

            toMergeCandidateDict, intermediatePool, generatedSeqNum = mergeCandidateClusters_new(
                toMergeCandidateDict, intermediatePool, scoreList, generatedSeqNum, initialDict, thresholdValue)

            print("generatedSeqNum now: ", generatedSeqNum)

            # check if algorithm should stop when merge score under threshold
            # if a score smaller than threshold, then it will break out when merging.
            # Hense, if the 'generatedSeqNum' equals than 'nameIdxStart', means that no any new generated cluster.
            # (if occurr a new cluster, generatedSeqNum will add one.)
            if(generatedSeqNum == nameIdxStart):
                residual = toMergeCandidateDict # output residual candidates.
                break # end algorithm

            nameIdxEnd = generatedSeqNum

            # Record clusters generated in this round
            for idx in range(nameIdxStart, nameIdxEnd):
                if roundInfos.get(roundCounter) is None:
                    roundProduct = [] #list()
                    roundProduct.append(intermediatePool[idx][1][0])
                    roundInfos[roundCounter] = roundProduct
                else:
                    roundInfos[roundCounter].append(intermediatePool[idx][1][0])

            roundCounter += 1

            with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(roundCounter)+'.pickle', 'wb') as mHandle:
                pickle.dump(toMergeCandidateDict, mHandle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(outputPath+ "pickle/" + 'intermediate_round'+str(roundCounter-1)+'.pickle', 'wb') as iHandle:
                pickle.dump(intermediatePool, iHandle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(outputPath+ "pickle/" + 'roundInfos_round'+str(roundCounter-1)+'.pickle', 'wb') as rHandle:
                pickle.dump(roundInfos, rHandle, protocol=pickle.HIGHEST_PROTOCOL)

#         print("-- Finish Clustering --")
    except:
        generatedSeqNum = generatedSeqNum
        roundCounter = roundCounter
        if(roundCounter != 1):
            with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(roundCounter)+'.pickle', 'rb') as mHandle:
                toMergeCandidateDict = pickle.load(mHandle)
            with open(outputPath+ "pickle/" + 'roundInfos_round'+str(roundCounter-1)+'.pickle', 'rb') as rHandle:
                roundInfos = pickle.load(rHandle)
            with open(outputPath+ "pickle/" + 'intermediate_round'+str(roundCounter-1)+'.pickle', 'rb') as iHandle:
                intermediatePool = pickle.load(iHandle)

        while(1):
            print("Round: ", roundCounter)
            if(len(toMergeCandidateDict) == 1):
                residual = toMergeCandidateDict # output residual candidates.
                break

            # calculate scoreList in candidate clusters
            scoreList = findMergeCandidateScoreList(toMergeCandidateDict, generatedSeqNum)

            # check and merge exactly the same candidates before merge clusters
            scoreList = checkExactlySameCandidates(scoreList)

            import os
            if not os.path.isdir(outputPath+str(roundCounter)+'TmpPickle/'):
                os.makedirs(outputPath+str(roundCounter)+'TmpPickle/')

            with open(outputPath+str(roundCounter)+'TmpPickle/' + 'scoreList.pickle', 'wb') as sListHandle:
                    pickle.dump(scoreList, sListHandle, protocol=pickle.HIGHEST_PROTOCOL)

            # generated Clusters in This Round:
            nameIdxStart = generatedSeqNum

            toMergeCandidateDict, intermediatePool, generatedSeqNum = mergeCandidateClusters_new(
                toMergeCandidateDict, intermediatePool, scoreList, generatedSeqNum, initialDict, thresholdValue)

            print("generatedSeqNum now: ", generatedSeqNum)

            # check if algorithm should stop when merge score under threshold
            # if a score smaller than threshold, then it will break out when merging.
            # Hense, if the 'generatedSeqNum' equals than 'nameIdxStart', means that no any new generated cluster.
            # (if occurr a new cluster, generatedSeqNum will add one.)
            if(generatedSeqNum == nameIdxStart):
                residual = toMergeCandidateDict # output residual candidates.
                break # end algorithm

            nameIdxEnd = generatedSeqNum

            # Record clusters generated in this round
            for idx in range(nameIdxStart, nameIdxEnd):
                if roundInfos.get(roundCounter) is None:
                    roundProduct = [] #list()
                    roundProduct.append(intermediatePool[idx][1][0])
                    roundInfos[roundCounter] = roundProduct
                else:
                    roundInfos[roundCounter].append(intermediatePool[idx][1][0])

            roundCounter += 1

            with open(outputPath+ "pickle/" + 'toMergeCandidate_round'+str(roundCounter)+'.pickle', 'wb') as mHandle:
                pickle.dump(toMergeCandidateDict, mHandle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(outputPath+ "pickle/" + 'intermediate_round'+str(roundCounter-1)+'.pickle', 'wb') as iHandle:
                pickle.dump(intermediatePool, iHandle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(outputPath+ "pickle/" + 'roundInfos_round'+str(roundCounter-1)+'.pickle', 'wb') as rHandle:
                pickle.dump(roundInfos, rHandle, protocol=pickle.HIGHEST_PROTOCOL)

#         print("-- Finish Clustering --")
    return intermediatePool, initialDict, roundInfos, residual

In [38]:
def clusterInitializedReps(initializedReps_dict, tag, outputPath, thresholdValue):
    intermediatePool = dict()
    roundInfos = dict()
    residual = None # used to save residual candidate when algorithm stop.
#     toMergeCandidateDict = testDict
    toMergeCandidateDict = initializedReps_dict # using residualRepsDict as toMergeCandidateDict (skip initialization)

    # initialDict = {clusterName : (originalName, initialLength)}
    initialDict = getInitialDict(toMergeCandidateDict)
#     with open(outputPath+'Pickle/' + 'tmpInit.pickle', 'wb') as sListHandle:
#             pickle.dump(initialDict, sListHandle, protocol=pickle.HIGHEST_PROTOCOL)
    
    roundProduct = [] #list()
    for key, value in initialDict.items():
        roundProduct.append(key)
    roundInfos[0] = roundProduct # record product in round 0 (i.e., initialization)
    
    generatedSeqNum = len(toMergeCandidateDict) # counter after initialize. Used to naming clusters.

#     print("-- Start Clustering --")
#     print("Threshold set =", thresholdValue)
    roundCounter = 1
    
    while(1):
#         print("Current Round : Round ", roundCounter)
        if(len(toMergeCandidateDict) == 1):
            residual = toMergeCandidateDict # output residual candidates.
            break

        # calculate scoreList in candidate clusters
        scoreList = findMergeCandidateScoreList(toMergeCandidateDict, generatedSeqNum)
        print("-- Finish scoring --")
#         print("ScoreList Len : ", len(scoreList))
        
        # check and merge exactly the same candidates before merge clusters
        scoreList = checkExactlySameCandidates(scoreList)
        print("-- Finish checking 100% same candidates --")
       
        if not os.path.isdir(outputPath+str(roundCounter)+'Pickle/'): os.makedirs(outputPath+str(roundCounter)+'Pickle/')
        with open(outputPath+str(roundCounter)+'Pickle/' + 'scoreList.pickle', 'wb') as sListHandle:
            pickle.dump(scoreList, sListHandle, protocol=pickle.HIGHEST_PROTOCOL)
        
        # generated Clusters in This Round:
        nameIdxStart = generatedSeqNum
        
        toMergeCandidateDict, intermediatePool, generatedSeqNum = mergeCandidateClusters_new(
            toMergeCandidateDict, intermediatePool, scoreList, generatedSeqNum, initialDict, thresholdValue)
        print("-- Finish merging clusters --")
        # check if algorithm should stop when merge score under threshold
        # if a score smaller than threshold, then it will break out when merging.
        # Hense, if the 'generatedSeqNum' equals than 'nameIdxStart', means that no any new generated cluster.
        # (if occurr a new cluster, generatedSeqNum will add one.)
        if(generatedSeqNum == nameIdxStart):
            residual = toMergeCandidateDict # output residual candidates.
            break # end algorithm
        
        nameIdxEnd = generatedSeqNum
        
        # Record clusters generated in this round
        for idx in range(nameIdxStart, nameIdxEnd):
            if roundInfos.get(roundCounter) is None:
                roundProduct = [] #list()
                roundProduct.append(intermediatePool[idx][1][0])
                roundInfos[roundCounter] = roundProduct
            else:
                roundInfos[roundCounter].append(intermediatePool[idx][1][0])
                
        roundCounter += 1
    print("-- Finish Clustering --")

    return intermediatePool, initialDict, roundInfos, residual

***

In [39]:
# import threading
# import time
# from threading import Thread


# exitFlag = 0

# class myThread (threading.Thread):
#     def __init__(self, threadID, name, counter):
#         threading.Thread.__init__(self)
#         self.threadID = threadID
#         self.name = name
#         self.counter = counter
#     def run(self):
#         print ("Starting " + self.name)
#         print_time(self.name, 1, self.counter)
#         print ("Exiting " + self.name)
# #         return -1
#     def join(self):
#         Thread.join(self)
#         return -1

# def print_time(threadName, counter, delay):
#     while counter:
# #         if exitFlag:
# #             threadName.exit()
#         time.sleep(delay)
#         print ("%s: %s" % (threadName, time.ctime(time.time())))
#         counter -= 1
# gg= [1,2,3,4,5]
# kk=['oo','ff','kk','yy','zz','11','22','44','666','1111','ggg','gjj','qqq','pppp','sss','oooo']
# # Create new threads
# c=0
# for v in kk:
#     for vv in gg:
#         thread1 = myThread(1, v, vv)
#     #     thread2 = myThread(2, v, 1)
#         k1= thread1.start()
#     #     thread2.start()
#         c+=1
#         c+
#         if c==10:
#             thread1.join()
#             c=0
#         time.sleep(1)

# # Start new Threads


# print ("Exiting Main Thread")